# Ver 0

In [ ]:

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import uncertainties as un
# Primer intento de hacer una función que automatice la medición de la distancias en base a los ROIs
mag="50X"
imagenes = ["_binario","_sombra","_sombra_binario"]
csv = ["_Cuadrado_X","_Cuadrado_Y","_Pelos_X"]
# Esta solo funciona para un tipo de imagen "A2_..." no pense en las otras imágenes
for efecto in imagenes:
    for zona in csv:
        # --- 1. Cargar imagen ---
        img = cv2.imread(f"../Pasantia SL/Mediciones/A2_{mag}{efecto}.tiff", cv2.IMREAD_GRAYSCALE)
        # --- 2. Cargar CSV con las ROIs ---
        rois = pd.read_csv(f"../Pasantia SL/Mediciones/A2_{mag}{zona}.csv") 
        # --- 3. Procesar cada ROI ---
        un_distancia_prom = []
        for i, roi in rois.iterrows():
            x, y, w, h = int(roi['X']), int(roi['Y']), int(roi['Width']), int(roi['Height'])
            # Recortar subimagen
            subimg = img[y:y+h, x:x+w]
            # --- 4. Detectar bordes ---
            blur = cv2.GaussianBlur(subimg, (3,3), 0)
            bordes = cv2.Canny(blur, 100, 131)
            
            distances = []
            h, w = bordes.shape
            # El recorte está horizontal en este caso
            if "X" in zona:
                for y in range(w):
                    row = bordes[:, y]
                    xs = np.where(row > 0)[0]  # Posiciones de bordes en esa fila
                    if xs.size >= 2:
                        # Distancia entre el borde más a la izquierda y el más a la derecha en esa fila
                        d = float(xs[-1] - xs[0])
                        distances.append(d)
            else:
                for y in range(h):
                    row = bordes[y,:]
                    xs = np.where(row > 0)[0]  # Posiciones de bordes en esa fila
                    if xs.size >= 2:
                        # Distancia entre el borde más a la izquierda y el más a la derecha en esa fila
                        d = float(xs[-1] - xs[0])
                        distances.append(d)  
                        

            plt.figure(figsize=(5,5))
            plt.imshow(bordes, cmap='gray')
            plt.title(f"ROI {i+1}: Bordes detectados")
            plt.show()
            un_distancia_prom.append(un.ufloat(np.mean(distances),np.std(distances)))
            
        un_np_distancia_prom = np.array(un_distancia_prom)
        print("#"*50)
        print(f"Esta es la zona:{zona.replace("_"," ")},con el efecto{efecto.replace("_"," ")}")
        display(pd.DataFrame(un_np_distancia_prom,columns=["Promedio +/- desv"]))
            


# Ver 3

In [ ]:

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import uncertainties as un
from scipy import stats as st

def detector_bordes(roi, img, img_color):
    """_summary_

    Args:
        roi (_type_): _description_
        img (_type_): _description_
        img_color (_type_): _description_

    Returns:
        _type_: _description_
    """
    x, y, w, h = map(int, [roi['X'], roi['Y'], roi['Width'], roi['Height']])
    subimg = img[y:y+h, x:x+w]
    blur = cv2.GaussianBlur(subimg, (7,7), 0)
    bordes = cv2.Canny(blur, 100, 131)
    # Pegamos los bordes sobre la imagen original
    img_color[y:y+h, x:x+w, 2] = np.maximum(img_color[y:y+h, x:x+w, 2], bordes)
    # Dibujamos un rectángulo que marque el ROI
    cv2.rectangle(img_color, (x, y), (x+w, y+h), (0,255,0), 4)
    return bordes,w,h,x,y

mag = "50X"
imagenes = ["","_binario", "_sombra", "_sombra_binario"]
csv = ["_Cuadrado_X", "_Cuadrado_Y", "_Pelos_X"]

for zona in csv:
    rois = pd.read_csv(f"../Pasantia SL/Mediciones/A2_{mag}{zona}.csv") 
    
    for i, roi in rois.iterrows():
        fig, axes = plt.subplots(1, len(imagenes), figsize=(15,5))
        fig.suptitle(f"ROI {i+1} | Zona: {zona.replace('_',' ')}", fontsize=16)
        
        for j, efecto in enumerate(imagenes):
            img = cv2.imread(f"../Pasantia SL/Mediciones/A2_{mag}{efecto}.tiff", cv2.IMREAD_GRAYSCALE)
            img_color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
            bordes,w,h,x,y = detector_bordes(roi, img, img_color)
            subimg=img[y:y+h,x:x+w]
            # --- Cálculo de puntos de borde para regresión ---
            hh, ww = bordes.shape
            if "Y" in zona:
                x_coords, y1, y2 = [], [], []
                for col in range(ww):
                    row = bordes[:, col]
                    xs = np.where(row > 0)[0]
                    if xs.size >= 2:
                        x_coords.append(col)
                        y1.append(float(xs[-1]))
                        y2.append(float(xs[0]))
                if len(x_coords) >= 2:
                    m1,b1,_,_,_ = st.linregress(x_coords, y1)
                    m2,b2,_,_,_ = st.linregress(x_coords, y2)
                    axes[j].imshow(bordes, cmap='gray')
                distances = np.array(y1)-np.array(y2)
                ang1 = np.atan(m1)*180/np.pi
                ang2 = np.atan(m2)*180/np.pi
            else:
                y_coords, x1, x2 = [], [], []
                for row in range(hh):
                    xs = np.where(bordes[row, :] > 0)[0]
                    if xs.size >= 2:
                        y_coords.append(row)
                        x1.append(float(xs[-1]))
                        x2.append(float(xs[0]))
                if len(y_coords) >= 2:
                    m1,b1,_,_,_ = st.linregress(y_coords, x1)
                    m2,b2,_,_,_ = st.linregress(y_coords, x2)
                    axes[j].imshow(bordes, cmap='gray')
                distances = np.array(x1)-np.array(x2)
                if m1==0 and m2==0:
                    ang1=90
                    ang2=90
                elif m2==0:
                    ang1=np.atan(1/m1)*180/np.pi
                    ang2=90
                elif m1==0:
                    ang1=90
                    ang2=np.atan(1/m2)*180/np.pi
                else:
                    ang1=np.atan(1/m1)*180/np.pi
                    ang2=np.atan(1/m2)*180/np.pi
            Delta = abs(ang2-ang1)
            un_distancia_prom = un.ufloat(np.mean(distances),np.std(distances))
            if Delta>90:
                Delta = abs(Delta-180)
            DF = {"Distancia +/- Err":f"{un_distancia_prom:.1uf}","Delta Ángulo":Delta,"Angulo Superior/Der":ang1,"Ángulo Inferior/Izq":ang2}
            print("#"*30,efecto.replace("_"," "),"#"*30)
            display(pd.DataFrame(data=DF,index=[1]))
            axes[j].set_title(efecto.replace("_"," ").capitalize())
        
        plt.tight_layout()
        plt.show()

# Ver 3.1

In [ ]:

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import uncertainties as un
def detector_bordes():
    x, y, w, h = map(int, [roi['X'], roi['Y'], roi['Width'], roi['Height']])
    subimg = img[y:y+h, x:x+w]
    blur = cv2.GaussianBlur(subimg, (3,3), 0)
    bordes = cv2.Canny(blur, 100, 131)
    # Pegamos los bordes sobre la imagen original
    img_color[y:y+h, x:x+w, 2] = np.maximum(img_color[y:y+h, x:x+w, 2], bordes)
    # Dibujamos un rectángulo que marque el ROI
    cv2.rectangle(img_color, (x, y), (x+w, y+h), (0,255,0), 4)
    return bordes

mag = "50X"
imagenes = ["_binario", "_sombra", "_sombra_binario"]
csv = ["_Cuadrado_X", "_Cuadrado_Y", "_Pelos_X"]

for efecto in imagenes:
    for zona in csv:
        img = cv2.imread(f"../Pasantia SL/Mediciones/A2_{mag}{efecto}.tiff", cv2.IMREAD_GRAYSCALE)
        rois = pd.read_csv(f"../Pasantia SL/Mediciones/A2_{mag}{zona}.csv") 
        
        # Convertimos la imagen a color para poder dibujar bordes en rojo
        img_color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        un_distancia_prom = []

        for i, roi in rois.iterrows():
            bordes = detector_bordes()
            
            # Calculamos distancia media
            distances = []
            hh, ww = bordes.shape
            if "Y" in zona:
                for col in range(ww):
                    row = bordes[:, col]
                    xs = np.where(row > 0)[0]
                    if xs.size >= 2:
                        distances.append(xs[-1] - xs[0])
            else:
                for row in range(hh):
                    xs = np.where(bordes[row, :] > 0)[0]
                    if xs.size >= 2:
                        distances.append(xs[-1] - xs[0])
            if len(distances) > 0:
                un_distancia_prom.append(un.ufloat(np.mean(distances), np.std(distances)))

        # Mostrar la imagen con todos los bordes marcados
        plt.figure(figsize=(8,8))
        plt.imshow(cv2.cvtColor(img_color, cv2.COLOR_BGR2RGB))
        plt.title(f"Zona: {zona.replace('_',' ')} | Efecto: {efecto.replace('_',' ')}")
        plt.show()

        un_np_distancia_prom = np.array([f"{val:.1uf}" for val in un_distancia_prom])
        print("#"*50)
        print(f"Zona: {zona.replace('_',' ')} | Efecto: {efecto.replace('_',' ')}")
        display(pd.DataFrame(un_np_distancia_prom, columns=["Promedio ± desv"]))


# Ver 3.14

In [ ]:

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import uncertainties as un
from scipy import stats as st
def detector_bordes():
    x, y, w, h = map(int, [roi['X'], roi['Y'], roi['Width'], roi['Height']])
    subimg = img[y:y+h, x:x+w]
    blur = cv2.GaussianBlur(subimg, (5,5), 0)
    bordes = cv2.Canny(blur, 100, 131)
    # Pegamos los bordes sobre la imagen original
    img_color[y:y+h, x:x+w, 2] = np.maximum(img_color[y:y+h, x:x+w, 2], bordes)
    # Dibujamos un rectángulo que marque el ROI
    cv2.rectangle(img_color, (x, y), (x+w, y+h), (0,255,0), 4)
    return bordes

mag = "50X"
imagenes = ["_binario", "_sombra", "_sombra_binario"]
csv = ["_Cuadrado_X", "_Cuadrado_Y", "_Pelos_X"]

for efecto in imagenes:
    for zona in csv:
        img = cv2.imread(f"../Pasantia SL/Mediciones/A2_{mag}{efecto}.tiff", cv2.IMREAD_GRAYSCALE)
        rois = pd.read_csv(f"../Pasantia SL/Mediciones/A2_{mag}{zona}.csv") 
        
        # Convertimos la imagen a color para poder dibujar bordes en rojo
        img_color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        un_distancia_prom = []
        print(f"#############{zona}#############")
        for i, roi in rois.iterrows():
            bordes = detector_bordes()
            
            # Calculamos distancia media
            distances = []
            hh, ww = bordes.shape
            
            if "Y" in zona:
                x,y1,y2=[],[],[]
                for col in range(ww):
                    row = bordes[:, col]
                    xs = np.where(row > 0)[0]
                    if xs.size >= 2:
                        x.append(col)
                        y1.append(float(xs[-1]))
                        y2.append(float(xs[0]))
                m1,b1,_,_,_=st.linregress(x,y1)
                m2,b2,_,_,_=st.linregress(x,y2)
                plt.plot(x,y1,"or")
                plt.plot(x,y2,"ob")
                plt.show()

            else:
                y,x1,x2=[],[],[]
                for row in range(hh):
                    xs = np.where(bordes[row, :] > 0)[0]
                    if xs.size>=2:
                        y.append(row)
                        x1.append(float(xs[-1]))
                        x2.append(float(xs[0]))
                m1,b1,_,_,_=st.linregress(y,x1)
                m2,b2,_,_,_=st.linregress(y,x2)
                plt.plot(x1,y,"or")
                plt.plot(x2,y,"ob")
                plt.show()

            if m1==0 and m2==0:
                ang1=90
                ang2=90
            elif m2==0:
                ang1=np.atan(1/m1)*180/np.pi
                ang2=90
            elif m1==0:
                ang1=90
                ang2=np.atan(1/m2)*180/np.pi
            else:
                ang1=np.atan(1/m1)*180/np.pi
                ang2=np.atan(1/m2)*180/np.pi
            Delta = abs(ang2-ang1)
            
            if Delta>90:
                print("Dm1",round(abs(180-Delta),1),"tita1",round(ang1,1),"tita2",round(ang2,1))
            else:
                print("Dm1",round(Delta,1),"tita1",round(ang1,1),"tita2",round(ang2,1))


# Otras funciones


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, AutoMinorLocator,AutoLocator

def configurar_estilo_global():
    """Configura el estilo global para todos los gráficos"""
    plt.rcParams.update({
    'figure.figsize': (6, 6),
    'font.size': 14,  # Aumentado de 12 a 14
    'axes.titlesize': 14,  # Aumentado
    'axes.labelsize': 11,  # Aumentado
    'legend.fontsize': 10,  # Aumentado
    'xtick.labelsize': 10,  # Aumentado
    'ytick.labelsize': 10,  # Aumentado
    'grid.color': '#CCCCCC',
    'grid.linestyle': '--',
    'grid.alpha': 0.7,
    'xtick.major.pad': 10,  # Espaciado para mejor legibilidad
    'ytick.major.pad': 10,  # Espaciado para mejor legibilidad
    'errorbar.capsize':3})

# Llamar esta función al inicio de tu script
configurar_estilo_global()
def buscar_datos_dict(nombre):
    """Busca datos en ambos diccionarios y retorna el diccionario correspondiente"""
    # Primero buscar en resultados_20
    for key in resultados_20.keys():
        if nombre in key or key in nombre:  # Búsqueda flexible
            return resultados_20, key
    
    # Luego buscar en resultados_50
    for key in resultados_50.keys():
        if nombre in key or key in nombre:  # Búsqueda flexible
            return resultados_50, key
    
    raise KeyError(f"Nombre '{nombre}' no encontrado en ningún diccionario")

def Picasso_automatico_dict():
    """Versión automática que busca en ambos diccionarios"""
    
    # Lista de pares predefinidos - usando las claves exactas de tus diccionarios
    pares = [
        # Par 1: Pelos 20X
        ["20 A2_1", "20 P_1"],
        # Par 2: Pelos 50X A2
        ["50 A2_1", "50 A2_2"],
        # Par 3: Pelos 50X P
        ["50 P_1", "50 P_2"],
        # Par 4: Cuadrado 50X (usando los mismos datos pero diferentes zonas)
        ["50 A2_1", "50 A2_1"],  # Mismo dataset, diferentes zonas
        # Par 5: Cuadrado 20X (usando los mismos datos pero diferentes zonas)
        ["20 A2_1", "20 A2_1"]   # Mismo dataset, diferentes zonas
    ]
    
    # Zonas específicas para cada par
    zonas_por_par = [
        # Par 1: Pelos 20X - ambas zonas de pelos
        [["_Pelos_X", "_Pelos_X2"], ["_Pelos_X", "_Pelos_X2"]],
        # Par 2: Pelos 50X A2 - ambas zonas de pelos
        [["_Pelos_X", "_Pelos_X2"], ["_Pelos_X", "_Pelos_X2"]],
        # Par 3: Pelos 50X P - ambas zonas de pelos
        [["_Pelos_X", "_Pelos_X2"], ["_Pelos_X", "_Pelos_X2"]],
        # Par 4: Cuadrado 50X - zonas X e Y
        [["_Cuadrado_X"], ["_Cuadrado_Y"]],
        # Par 5: Cuadrado 20X - zonas X e Y
        [["_Cuadrado_X"], ["_Cuadrado_Y"]]
    ]
    
    nombres_descriptivos = [
        "Pelos 20X: A2_1 vs P_1",
        "Pelos 50X: A2_1 vs A2_2", 
        "Pelos 50X: P_1 vs P_2",
        "Cuadrado 50X: Eje X vs Eje Y",
        "Cuadrado 20X: Eje X vs Eje Y"
    ]
    
    for i, (par, zonas_par, nombre_desc) in enumerate(zip(pares, zonas_por_par, nombres_descriptivos)):
        print(f"Plotando {i+1}: {nombre_desc}")
        print(f"  Datos: {par[0]} vs {par[1]}")
        print(f"  Zonas: {zonas_par[0]} vs {zonas_par[1]}")
        
        fig, ax = plt.subplots(figsize=(12, 8))
        
        # Plotear ambos elementos del par
        for j, (nombre, zonas) in enumerate(zip(par, zonas_par)):
            try:
                # Buscar en qué diccionario están los datos
                dict_resultados, clave_real = buscar_datos_dict(nombre)
                
                for zona in zonas:
                    try:
                        # Acceder a los datos: dict_resultados[clave_real][zona]
                        datos_zona = dict_resultados[clave_real][zona]
                        
                        X = datos_zona["distancia del centro"].to_numpy()
                        Y_un = datos_zona["px/mm"].to_numpy()
                        Y = unumpy.nominal_values(Y_un)
                        err = unumpy.std_devs(Y_un)
                        
                        # Crear label descriptivo
                        label_name = f"{clave_real} {zona.replace('_', ' ')}"
                        ax.errorbar(X, Y, yerr=err, fmt='.', capsize=3, capthick=1, 
                                   label=label_name)
                        
                    except KeyError as e:
                        print(f"  ⚠️  No se pudo acceder a {clave_real}[{zona}]: {e}")
                        
            except KeyError as e:
                print(f"❌ Error: {e}")
                continue
        
        # Configuración del gráfico
        ax.set_xlabel("Distancia al centro de la imagen [px]")
        ax.set_ylabel("Relación [px/mm]")
        ax.legend(fontsize=10)
        ax.set_title(nombre_desc, pad=20)
        
        # Configuración de ejes y notación científica
        ax.xaxis.set_minor_locator(AutoMinorLocator(10))
        ax.yaxis.set_minor_locator(AutoMinorLocator(10))
        ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
        
        ax.grid(which='major', linestyle='--', alpha=0.7)
        ax.grid(which='minor', linestyle=':', alpha=0.5)
        
        plt.tight_layout()
        plt.show()
        print("-" * 50)

# Uso:
Picasso_automatico_dict()
## Pasar la parte de los estilos a la version de abajo
import numpy.polynomial.polynomial as poly
from scipy import stats
def U(x):
    return np.array([1,x,x**2])
def Covar_mat(x,y,s):
    A= poly.polyvander(x, 2)
    W=np.diag(s**-2)
    N=A.T @ W @ A
    b=A.T@W@y
    a=np.linalg.solve(N,b)
    res=y-A@a
    chis2=np.dot(s**-2,res**2)
    s2=chis2/(len(x)-3)
    return s2 * np.linalg.inv(N)
def sigma(r,Cov):
    if isinstance(r,float):
        return np.sqrt(U(r).T @ Cov @ U(r))
    else:
        return np.array([np.sqrt(U(a).T @ Cov @ U(a)) for a in r])
def f(x,v):
    return v[0]*x**0+v[1]*x+v[2]*x**2
def estadisticas(df,nombre,zona):
    if len(nombre)==1:
        x = df[nombre[0]][zona[0]]["distancia del centro"].to_numpy()
        y_un = df[nombre[0]][zona[0]]["px/mm"].to_numpy()
        y = unumpy.nominal_values(y_un)
        err = unumpy.std_devs(y_un)
        wheights = np.power(err,-1)
        popt = np.polynomial.polynomial.polyfit(x, y, 2, rcond=None, full=False, w=wheights)
        matriz_Covarianza=Covar_mat(x,y,err)
        y_fit = unumpy.uarray(f(x,popt),sigma(x,matriz_Covarianza))
        residuos = y_un-y_fit
        ss_residuos = np.sum(residuos**2)
        ss_total = np.sum((y_un-np.mean(y))**2)
        r2 = 1 - ss_residuos/ss_total
        print(r2)
        n = len(x)
        grados_libertad = n - 3
        t_critico = stats.t.ppf(0.975, grados_libertad) # 95% confianza

        print(f"\nValor crítico t(0.025, {grados_libertad}) = {t_critico:.4f}")

        # Errores estándar (raíz de diagonal de covarianza)
        errores = np.sqrt(np.diag(matriz_Covarianza))*t_critico
        popt_un=[]
        for i in range(3):
            popt_un.append(un.ufloat(popt[i],errores[i]))
        print(f"####################\nEl mejor ajuste es:\ny=({popt_un[0]:.1u})  +  ({popt_un[1]:1u})*r  +  ({popt_un[2]:.1u})*r²")
        r_n = unumpy.nominal_values(residuos)
        r_s = unumpy.std_devs(residuos)
        plt.errorbar(x,r_n,yerr=r_s, fmt='o', label='Residuos')
        plt.show()
        fig,ax =plt.subplots()
        ax.errorbar(x, y, yerr=err, fmt='o', label='Datos')
        X = np.linspace(0,max(x),300)
        ax.plot(X, f(X,popt), label='Ajuste')
        ax.fill_between(X,  f(X,popt) - sigma(X,matriz_Covarianza),
                            f(X,popt) + sigma(X,matriz_Covarianza),
                            alpha=0.3, label='IC ajuste')
        ax.legend()
        plt.show()

## Quedo en desuso

In [ ]:
import pandas as pd
from uncertainties import unumpy
import uncertainties as un
import numpy as np
import matplotlib.pyplot as plt
import math

def pretty_unc(x, sig=1):
    """
    Devuelve una cadena del tipo '7300+/-300' para un ufloat o lista de ufloats.
    - x: ufloat, lista o np.array de ufloats
    - sig: cantidad de cifras significativas para la incertidumbre (default: 1)
    """
    
    def _format_single(val):
        v, e = val.n, val.s
        if e == 0:
            return f"{v}"
        
        # Exponente del error
        exp = int(math.floor(math.log10(abs(e))))
        # Redondear error a 'sig' cifras significativas
        err_rounded = round(e, -exp + (sig - 1))
        # Redondear valor al mismo orden
        val_rounded = round(v, -exp + (sig - 1))
        
        # Si son enteros, mostrarlos sin decimales
        if err_rounded.is_integer() and val_rounded.is_integer():
            return f"{int(val_rounded)}+/-{int(err_rounded)}"
        else:
            return f"{val_rounded}+/-{err_rounded}"
    
    # Si es lista o array, aplicar a cada elemento
    if isinstance(x, (list, np.ndarray)):
        return np.array([_format_single(xi) for xi in x])
    else:
        return _format_single(x)
def extraer_datos(DF:pd.DataFrame,clave:str,valor:str,columna="distance_ufloat",mag=50):

    Resultados = DF[DF[clave]==valor]
    Datos_np = Resultados[columna].to_numpy()
    if clave=="zona":
        if valor=="_Pelos_X" or valor=="_Pelos_X2":
            medida = 0.001567
        elif valor=="_Cuadrado_X":
            medida = 0.03054
        elif valor=="_Cuadrado_Y":
            medida = 0.03093
        k = [un.ufloat_fromstr(val)/medida for val in Datos_np]
        Resultados = Resultados.assign(**{"px/mm":pd.Series(k,index=Resultados.index).values})
        if mag==50:
            L=2980 #px
        else:
            L=3040 #px
        Xpos2 = Resultados["X_centro"].map(lambda x: x-L/2)**2
        Ypos2 = Resultados["Y_centro"].map(lambda x: -x+L/2)**2
        r2 = (Xpos2+Ypos2)
        r=r2.map(lambda x: x**0.5)
        Resultados = Resultados.assign(**{"distancia del centro":pd.Series(r,index=Resultados.index).values})
        Resultados.drop(["X_centro","Y_centro","efecto","distance_mean","distance_std","zona","roi"], inplace=True, axis=1)
        return Resultados
    

    
### Pelos 20X ###
k_Pelos_20X_A2 = extraer_datos(df_20X_A2,"zona","_Pelos_X",mag=20)
k_Pelos_20X_P  = extraer_datos(df_20X_P,"zona","_Pelos_X",mag=20)
### Pelos 50X ###
k_Pelos_50X_A2 = extraer_datos(df_50X_A2,"zona","_Pelos_X")
k_Pelos_50X_2_A2 = extraer_datos(df_50X_2_A2,"zona","_Pelos_X2")
## Pelos 50X ##
k_Pelos_50X_P  = extraer_datos(df_50X_P ,"zona","_Pelos_X") ## El 3 es un outlier
k_Pelos_50X_P2 = extraer_datos(df_50X_P2,"zona","_Pelos_X") ## El 1 es medio un outlier
### Cuadrado 50X A2 ###
k_CuadradoX_50X_A2 = extraer_datos(df_50X_A2,"zona","_Cuadrado_X")
k_CuadradoY_50X_A2 = extraer_datos(df_50X_A2,"zona","_Cuadrado_Y")
### Cuadrado 20X  A2 ###
k_CuadradoX_20X_A2 = extraer_datos(df_20X_A2,"zona","_Cuadrado_X",mag=20)
k_CuadradoY_20X_A2 = extraer_datos(df_20X_A2,"zona","_Cuadrado_Y",mag=20)
#3040->20X
#2980->50X
plot=0
k_Data_20 = {"20 A2_1":df_20X_A2,"20 P_1":df_20X_P}
k_Data_50 = {"50 A2_1":df_50X_A2,"50 A2_2":df_50X_2_A2,"50 P_1":df_50X_P,"50 P_2":df_50X_P2}

